In [6]:
!pip install Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.5 MB/s eta 0:00:00


In [7]:
!pip install fuzzywuzzy

In [ ]:
"""

Future Scope we can replace this embedding model with our fine tuned embedding model, may be this one or siamese or BERT or Siamese + BERT
EMBEDDING MODELS (SEMANTIC TEXTUAL SIMILARITY)
name 1 --> Preprocessing 1 --> Embedding Model (Open/Closed Source) --> Embedding 1 --> Cosine Similarity to get Similarity Score  --> weightage --> 0.4 (40% Weightage)
name 2 --> Preprocessing 2 --> Embedding Model (Open/Close Source) --> Embedding 2 -->

LEVENTHESIN DISTANCE (STRING MATCHING)
name 1 --> Preprocessing 1 --> Leventhesisn Distance --> Similarity Score -->  weightage --> 0.2 (20% Weightage) (USE- STRING MATCHING)
name 2 --> Preprocessing 2 -->

PHONETIC SOUND (SH->D)
name 1 --> Preprocessing 1 --> SoundDex/Metaphone --> Similarity Score -->  weightage --> 0.2 (20% Weightage) (USE- FOR THE NAMES HAVING SAME SOUNDS)
name 2 --> Preprocessing 2 --> SoundDex/Metaphone

JACCARD SIMILARITY (NUMBER OF OCCCURENCES)
name 1 --> Preprocessing 1 --> Jaccard Similarity --> Similarity Score -->  weightage --> 0.1 (10% Weightage) (USE- STRING MATCHING)
name 2 --> Preprocessing 2 --> Jaccard Similarity

FUZZY Wuzzy (RATIO)
name 1 --> Preprocessing 1 --> Fuzzy Matching --> Similarity Score -->  weightage --> 0.1 (10% Weightage)
name 2 --> Preprocessing 2 --> Fuzzy Matching -->


name 1 name2 --> preprocessing
fuzzy wuzzy --> score (matchScore)

if matchScore>80:
  Aman Sharma vs Raman Sharma hight
  Amar Sharma vs Aman Sharma



"""

In [1]:
import re

def remove_parent_spouse_name(text):
    """Remove relationship markers like 'D/O', 'S/O', 'daughter of', 'son of', etc."""
    return re.sub(r'\s*(?:s/o|d/o|w/o|so|do|wo|daughter of|son of|wife of|husband of|daughter|son|child of)\s*[\w\s,.]*$', '', text, flags=re.IGNORECASE).strip()

# Example usage
name = "SauaravSonOfManmeet"
cleaned_name = remove_parent_spouse_name(name)
print(cleaned_name)  # Output: "Riya"

Sauarav


In [45]:
COMMON_MUSLIM_SALUTATIONS_MOHAMMAD_REGEX = r"\b(mohammad|mohammed|muhamed|mohd|mohamed|mohamad|muhamad|muhammad|muhammed|muhammet|mohamud|mohummad|mohummed|mouhamed|muhamaad|mohammod|mouhamad|mo|md|mahmood|mahmud|ahmad|ahmed|hameed|hamid|hammed|mahd|mahmod|mohd|mouhammed|mohamad|muhmood|mohhammed|muhmamed|mohmed|mohmat|muhmat|mu|m|shaikh|mo)\b"

In [47]:
def remove_common_muslim_variations(text):
    """Remove common variations of 'Mohammad' and similar names."""
    return re.sub(COMMON_MUSLIM_SALUTATIONS_MOHAMMAD_REGEX, '', text, flags=re.IGNORECASE).strip()
# MD YOUSUF	MDYOUSUF  KHASEEMSAB
# MOHD SALEEM	SALIM ISLAM SHAIKH
name = "MO BASIM"
cleaned_name = remove_common_muslim_variations(name)
print(cleaned_name)  # Output: "Riya"

BASIM


In [ ]:
# moombatiye

In [33]:
def remove_extra_spaces(name):
    """Remove extra spaces between words, allowing only one space."""
    return re.sub(r'\s+', ' ', name).strip()


# Example usage:
example_name = "Singh  Manmeet"
processed_name = remove_extra_spaces(example_name)
print(processed_name)  # Output: "John Doe"

Singh Manmeet


In [55]:
## Main Code
from transformers import AutoModel, AutoTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
from Levenshtein import distance as levenshtein_distance
from fuzzywuzzy import fuzz
import jellyfish
import re

# Regular expressions for different patterns
SPECIAL_CHAR_DOT_REGEX = r"[.]"
SPECIAL_CHARS_REGEX = r"[-+.^:,_/\s]+"  # Allowing whitespace as a separator
SALUTATION_REGEX = r"^(shree|shri|miss|smt|mrs|mr|ms|dr|master|hon|sir|madam|prof|capt|major|rev|fr|br)\s*"
PARENT_SPOUSE_NAME_REGEX = r"(?:\s*(?:s/o|d/o|w/o|so|do|wo|daughter of|son of|wife of|husband of)\s*)"
COMMON_MUSLIM_SALUTATIONS_MOHAMMAD_REGEX = r"\b(mohammad|mohammed|muhamed|mohd|mohamed|mohamad|muhamad|muhammad|muhammed|muhammet|mohamud|mohummad|mohummed|mouhamed|muhamaad|mohammod|mouhamad|mo|md|mahmood|mahmud|ahmad|ahmed|hameed|hamid|hammed|mahd|mahmod|mohd|mouhammed|mohamad|muhmood|mohhammed|muhmamed|mohmed|mohmat|muhmat|mu|m|shaikh|mo)\b"
COMMON_MIDDLE_NAMES_REGEX = r"\b(kumar|kumari|singh|raj|dev|lal|prasad|rani|bai|khan|kumaravel|kanya|krishna|eshwar)\b"
COMMON_LAST_NAMES_REGEX = r"\b(gupta|sharma|verma|rawat|pandey|yadav|mehta|patel|kapoor|malhotra|choudhary|chaudhary|jha|yadav|bhat|nath)\b"
LAST_NAMES_AGARWAL_VARIANTS_REGEX = r"\b(aggarwal|agrawal|agarwal|aggrawal|agarwalla|agarwal)\b"

def convert_to_lower(name):
    return name.lower()

def replace_adjacent_duplicates(value):
    if isinstance(value, str):
        return re.sub(r'(.)\1+', r'\1', value)
    else:
        return value

def replace_characters(name):
    replacements = {'e': 'i', 'j': 'z', 'v': 'w', 'q': 'k'}
    for old, new in replacements.items():
        name = name.replace(old, new)
    return name

def replace_bigrams(name):
    replacements = {'ph': 'f', 'gh': 'g', 'th': 't', 'kh': 'k', 'dh': 'd', 'ch': 'c', 'sh': 's', 'au': 'o',
                    'bh': 'b', 'ks': 'x', 'ck': 'k', 'ah': 'h', 'wh': 'w', 'wr': 'r'}
    for old, new in replacements.items():
        name = name.replace(old, new)
    return name

def remove_extra_spaces(name):
    """Remove extra spaces between words, allowing only one space."""
    return re.sub(r'\s+', ' ', name).strip()


def remove_consonant_a(name):
    consonants = 'bcdfghjklmnpqrstvwxyz'
    new_name = ''
    for i in range(len(name)):
        if i > 0 and name[i] == 'a' and name[i - 1].lower() in consonants:
            continue
        new_name += name[i]
    return new_name

def remove_special_characters(text):
    """Remove various special characters defined in regex patterns."""
    text = re.sub(SPECIAL_CHAR_DOT_REGEX, '', text)
    text = re.sub(SPECIAL_CHARS_REGEX, '', text)
    return text.strip()

def remove_salutations(text):
    """Remove salutations based on regex patterns for titles."""
    return re.sub(SALUTATION_REGEX, '', text, flags=re.IGNORECASE).strip()

def remove_parent_spouse_name(text):
    """Remove relationship markers like 'D/O', 'S/O', 'daughter of', 'son of', etc."""
    # Extend the regex to match various phrases and handle case insensitivity
    return re.sub(r'\s*(?:s/o|d/o|w/o|so|do|wo|daughter of|son of|wife of|husband of|daughter|son|child of)\s*[\w\s,.]*$', '', text, flags=re.IGNORECASE).strip()

def remove_common_muslim_variations(text):
    """Remove common variations of 'Mohammad' and similar names."""
    return re.sub(COMMON_MUSLIM_SALUTATIONS_MOHAMMAD_REGEX, '', text, flags=re.IGNORECASE).strip()

def remove_common_middle_names(text):
    """Remove common middle names such as 'Kumar' and 'Singh'."""
    return re.sub(COMMON_MIDDLE_NAMES_REGEX, '', text, flags=re.IGNORECASE).strip()

def remove_common_last_names(text):
    """Remove common last names like 'Gupta' and 'Sharma'."""
    return re.sub(COMMON_LAST_NAMES_REGEX, '', text, flags=re.IGNORECASE).strip()

def remove_agarwal_variants(text):
    """Remove variants of the surname 'Agarwal'."""
    return re.sub(LAST_NAMES_AGARWAL_VARIANTS_REGEX, '', text, flags=re.IGNORECASE).strip()

def remove_stop_words(text):
    stop_words = [
        'devi', 'dei', 'debi',
        'kumar', 'kumaar', 'kumari', 'kumaari', 'kmr', 'kumr',
        'bhai', 'bhau', 'bai',
        'ben',
        'singh', 'kaur',
        'Md', 'Mohd', 'Mohammad', 'Mohamad'
    ]

    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

def preprocess(name):
    """Process a name through all the defined normalization steps."""
    name = remove_salutations(name)
    name = remove_parent_spouse_name(name)
    name = remove_common_muslim_variations(name)
    name = remove_common_middle_names(name)
    name = remove_common_last_names(name)
    name = remove_agarwal_variants(name)
    name = convert_to_lower(name)
    name = replace_adjacent_duplicates(name)
    name = replace_characters(name)
    name = replace_bigrams(name)
    name = remove_consonant_a(name)
    name = remove_special_characters(name)
    name = remove_stop_words(name)
    name=remove_extra_spaces(name)
    return name

# Load pre-trained models
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
model = AutoModel.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings

def calculate_cosine_similarity(embedding1, embedding2):
    return cosine_similarity(embedding1, embedding2).item()

def calculate_levenshtein_similarity(name1, name2):
    lev_distance = levenshtein_distance(name1, name2)
    max_len = max(len(name1), len(name2))
    return (max_len - lev_distance) / max_len if max_len > 0 else 1.0

def calculate_phonetic_similarity(name1, name2):
    soundex1 = jellyfish.soundex(name1)
    soundex2 = jellyfish.soundex(name2)
    return jellyfish.jaro_winkler_similarity(soundex1, soundex2)

def calculate_jaccard_similarity(name1, name2):
    set1 = set(name1)
    set2 = set(name2)
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union) if union else 1.0

def name_match(name1, name2):
    name1_processed = preprocess(name1)
    name2_processed = preprocess(name2)

    embedding1 = get_embedding(name1_processed)
    embedding2 = get_embedding(name2_processed)
    embedding_similarity = calculate_cosine_similarity(embedding1, embedding2)
    print("Printing the embedding Similarity ", embedding_similarity)

    levenshtein_similarity = calculate_levenshtein_similarity(name1_processed, name2_processed)
    print("Printing the Levenshtein Similarity ", levenshtein_similarity)

    phonetic_similarity = calculate_phonetic_similarity(name1_processed, name2_processed)
    print("Printing the Phonetic Similarity ", phonetic_similarity)

    jaccard_similarity = calculate_jaccard_similarity(name1_processed, name2_processed)
    print("Printing the Jaccard Similarity ", jaccard_similarity)

    fuzzy_ratio = fuzz.ratio(name1_processed, name2_processed) / 100.0
    fuzzy_partial_ratio = fuzz.partial_ratio(name1_processed, name2_processed) / 100.0
    fuzzy_token_sort_ratio = fuzz.token_sort_ratio(name1_processed, name2_processed) / 100.0
    fuzzy_token_set_ratio = fuzz.token_set_ratio(name1_processed, name2_processed) / 100.0

    fuzzy_similarity = (fuzzy_ratio + fuzzy_partial_ratio + fuzzy_token_sort_ratio + fuzzy_token_set_ratio) / 4.0
    print("Printing the Fuzzy Similarity ", fuzzy_similarity)

    weight_embedding = 0.4  # Important for capturing semantic similarities
    weight_levenshtein = 0.2  # Useful for identifying minor typographical errors
    weight_phonetic = 0.15  # Helpful for matching similar-sounding names
    weight_jaccard = 0.15  # Good for measuring the overlap of characters
    weight_fuzzy = 0.1  # Useful for broader string matching but can be less precise

    final_score = (
        embedding_similarity * weight_embedding +
        levenshtein_similarity * weight_levenshtein +
        phonetic_similarity * weight_phonetic +
        jaccard_similarity * weight_jaccard +
        fuzzy_similarity * weight_fuzzy
    )

    return final_score


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [57]:
# MOHD JAFAR	JAFAR ENTERPRISES
# Example usage
# Mr. MOHD MUSHTAQE SHAIKH	MOHD MUSHTAQUE AHMED
# AQEEL AHMED JAHANGEER MOHD	Mr. MOHD  AQEEL  AHMAD
name1 = "AQEEL AHMED JAHANGEER MOHD"
name2 = "Mr. MOHD  AQEEL  AHMAD"
# MOHD BASIM	MOBASIM

final_score = name_match(name1, name2)

# Check if final_score is greater than 75%
if final_score > 0.80:
    print("Name is Matching")
    print(f"Final Similarity Score: {final_score:.2f} (Threshold: 0.75)")
else:
    print("Name is Not Matching")
    print(f"Final Similarity Score: {final_score:.2f} (Threshold: 0.75)")

Printing the embedding Similarity  0.8607734441757202
Printing the Levenshtein Similarity  0.4
Printing the Phonetic Similarity  0.8833333333333334
Printing the Jaccard Similarity  0.4444444444444444
Printing the Fuzzy Similarity  0.6774999999999999
Name is Not Matching
Final Similarity Score: 0.69 (Threshold: 0.75)


In [ ]:
"""
91 with BAAI/bge-base-en  (English Language Only)
86 with BAAI/bge-large-en-v1.5 (English Language Only)
91 with BAAI/bge-m3 (Multilingual)
90 with BAAI/bge-base-en (English Language Only)
0.93 with sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2

"""